In [2]:
import pandas as pd
import numpy as np
import io
import math
import random
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
import cmapy
import nlpaug
import nlpaug.augmenter.audio as naa
from scipy.signal import butter, lfilter
import torch
from tqdm import tqdm
import os

In [3]:
audio_train=pd.read_csv('Temp_train_audio.csv')
audio_train.head(4)

,Unnamed: 0,Patient_ID,Disease,Recording_index,Chest_Location,Acquisition_Mode,Recording_equipment,Respiratory_cycles,Normal_cycles,Wheeze_cycles,Crackle_cycles,Both,Filename,Train/Test
0,13,107,NaN,2b3,Al,mc,AKGC417L,8,1,0,7,0,107_2b3_Al_mc_AKGC417L.txt,train
1,14,107,NaN,2b3,Ar,mc,AKGC417L,8,0,0,1,7,107_2b3_Ar_mc_AKGC417L.txt,train
2,15,107,NaN,2b3,Ll,mc,AKGC417L,8,0,0,8,0,107_2b3_Ll_mc_AKGC417L.txt,train
3,16,107,NaN,2b3,Lr,mc,AKGC417L,8,1,0,7,0,107_2b3_Lr_mc_AKGC417L.txt,train


In [4]:
sample_rate=4000
fs=30000
data_dir='AudioFiles2/'
desired_length=8
n_mels = 64
nfft = 256
hop = nfft//2
f_max = 2000
train_flag=1
lowcut=50
highcut=2000

In [5]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b,a, data)
    return y

In [6]:
#slicing into breath cycles
def slice_data(start, end, raw_data, sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    y=butter_bandpass_filter(raw_data[start_ind: end_ind], lowcut, highcut, fs, order=5)
    return y

In [7]:
#getting label of each cycles
def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

In [8]:
#getting breath cycle samples
def get_sound_samples(file_name, data_dir, sample_rate):
    sample_data = [file_name]
    data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
    recording_annotations=pd.read_csv(data_dir+file_name+'.txt', sep='\t', header=None)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row[0]
        end = row[1]
        crackles = row[2]
        wheezes = row[3]
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
    return sample_data

In [9]:
#creating list of breath cycles with respective labels
filenames=audio_train['Filename']
filenames_with_labels=[]
cycle_list = []
classwise_cycle_list = []
for idx, filename in tqdm(enumerate(filenames)):
            file_name=filename.split('.')[0]
            print(file_name+" going on "+str(idx)+" out of "+str(len(filenames)))
            data = get_sound_samples(file_name, data_dir, sample_rate)
            cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, 0) for cycle_idx, d in enumerate(data[1:])]
            cycle_list.extend(cycles_with_labels)
            for cycle_idx, d in enumerate(cycles_with_labels):
                filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
                classwise_cycle_list.append((d[0],d[1]))

0it [00:00, ?it/s]

107_2b3_Al_mc_AKGC417L going on 0 out of 41


1it [00:02,  2.38s/it]

107_2b3_Ar_mc_AKGC417L going on 1 out of 41


2it [00:03,  1.36s/it]

...

110_1b1_Pr_sc_Meditron going on 29 out of 41
110_1p1_Al_sc_Meditron going on 30 out of 41
110_1p1_Ll_sc_Meditron going on 31 out of 41
110_1p1_Lr_sc_Meditron going on 32 out of 41
110_1p1_Pr_sc_Meditron going on 33 out of 41
111_1b2_Tc_sc_Meditron going on 34 out of 41


35it [00:21,  3.48it/s]

111_1b3_Tc_sc_Meditron going on 35 out of 41


41it [00:22,  1.81it/s]

112_1b1_Ar_sc_Meditron going on 36 out of 41
112_1b1_Lr_sc_Meditron going on 37 out of 41
112_1p1_Ll_sc_Litt3200 going on 38 out of 41
112_1p1_Pl_sc_Litt3200 going on 39 out of 41
112_1p1_Pr_sc_Litt3200 going on 40 out of 41


In [ ]:
largest=0
file=""
cycle_id=0
id=0
for i in range(4142):
    if largest<cycle_list[i][0].shape[0]:
        largest=cycle_list[i][0].shape[0]
        file=cycle_list[i][2]
        cycle_id=cycle_list[i][3]
        id=i
print(largest, file, cycle_id, id)

In [10]:
largest_stft=librosa.stft(cycle_list[id][0], n_fft=512, hop_length = 64)
largest_stft.shape

TypeError: list indices must be integers or slices, not builtin_function_or_method

In [11]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [12]:
#creating sliced and padded data = audio_data
stft_data=[]
for idx, sample in enumerate(cycle_list):
            print(str(idx)+" out of "+str(len(cycle_list)))
            stft=librosa.stft(sample[0], n_fft=255, hop_length = 512)
            #print(stft.shape, sample[0].shape)
            stft_data.append(padding(stft, 257, 1011))

0 out of 365
1 out of 365
2 out of 365
3 out of 365
4 out of 365
5 out of 365
6 out of 365
7 out of 365
8 out of 365
9 out of 365
10 out of 365
11 out of 365
12 out of 365
13 out of 365
14 out of 365
15 out of 365
16 out of 365
17 out of 365
18 out of 365
19 out of 365
20 out of 365
21 out of 365
22 out of 365
23 out of 365
24 out of 365
25 out of 365
26 out of 365
27 out of 365
28 out of 365
29 out of 365
30 out of 365
31 out of 365
32 out of 365
33 out of 365
34 out of 365
35 out of 365
36 out of 365
37 out of 365
38 out of 365
39 out of 365
40 out of 365
41 out of 365
42 out of 365
43 out of 365
44 out of 365
45 out of 365
46 out of 365
47 out of 365
48 out of 365
49 out of 365
50 out of 365
51 out of 365
52 out of 365
53 out of 365
54 out of 365
55 out of 365
56 out of 365
57 out of 365
58 out of 365
59 out of 365
60 out of 365
61 out of 365
62 out of 365
63 out of 365
64 out of 365
65 out of 365
66 out of 365
67 out of 365
68 out of 365
69 out of 365
70 out of 365
71 out of 365
72

In [13]:
train_dataset=[]
for index in range(len(stft_data)):
    stft=stft_data[index]
    label = classwise_cycle_list[index][1]
    train_dataset.append((stft, label))

In [15]:
X_train=[]
y_train=[]
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0])
    y_train.append(train_dataset[i][1])  
    
np.save('X_train_STFT_small', np.concatenate(X_train, axis=0).reshape(len(train_dataset), 257, 1011, 1))
np.save('y_train_STFT_small', y_train)